In [6]:
import bluemath_tk.wrappers._base_wrappers

import pandas as pd
import itertools
import os

templates_dir = "/gpfs/work3/0/ai4nbs/hurry_wave/north_sea/04_modelruns/sensitivity_december2023_TEMPLATE"
output_dir = "/gpfs/work3/0/ai4nbs/hurry_wave/north_sea/04_modelruns/SENSITIVITY/12December2023"

length_x = 0.05 * 440 # # Length of the domain in x-direction [degrees]
length_y = 0.033333333 * 510 # Length of the domain in y-direction [degrees]

nruns = 11  # 1 default run + 8 sensitivity runs

# defaults (these are the values for the 1st run)
defaults = {
    "dt": 300,             # seconds
    "dx": 0.05,            # degrees
    "dy": 0.03333333,      # degrees
    "dmx1": 0.2,           # default from run plan
    "dmx2": 1e-5,          # kept from original
    "gammajsp": 3.3,
    "winddrag": "zijlema",
    "fbed": 0.019,
    "vmax_zijlema": 50,    # added to reflect sensitivity plan
    "nsigma": 12,
    "ntheta": 36,
    "freqmin": 0.04,       # Hz
    "freqmax": 0.5,        # Hz
}

print("initialize fields with default repeated for all runs")
# initialize fields with default repeated for all runs
fields = {k: [v] * nruns for k, v in defaults.items()}
print("FINISHED initialize fields with default repeated for all runs")

# Apply the run plan changes (runs 2..11 correspond to the table entries;
# index 0 is the default run)
# Run 2 (table 0001): dt -> 10
fields["dt"][1] = 10

# Run 4 (table 0003): dx, dy -> 0.25
fields["dx"][2] = 0.25
fields["dy"][2] = 0.25

fields['freqmin'][3] = 0.01
fields['freqmax'][3] = 1

fields['nsigma'][4] = 24

fields['ntheta'][5] = 72

# Run 5 (table 0004): vmax_zijlema -> 30
fields["vmax_zijlema"][6] = 30

# Run 10 (table 0009): winddrag -> 'wu'
fields["winddrag"][7] = "wu"

# Run 6 (table 0005): fbed -> 0.001
fields["fbed"][8] = 0.001

# Run 7 (table 0006): fbed -> 0.1
fields["fbed"][9] = 0.1

# Run 11 (table 0010): gammajsp -> 3
fields["gammajsp"][10] = 3.0

# Get the keys (column names) and the cartesian product of the values
keys = fields.keys()
# combinations = list(itertools.product(*fields.values()))

print("got column names")



initialize fields with default repeated for all runs
FINISHED initialize fields with default repeated for all runs
got column names


In [7]:
print(fields['dt'])

[300, 10, 300, 300, 300, 300, 300, 300, 300, 300, 300]


In [8]:
# Create the DataFrame
df = pd.DataFrame(fields, columns=keys)
df["mmax"] = length_x / df["dx"]
df["nmax"] = length_y / df["dy"]

print("adjustedmmax")

# Convert the DataFrame back to a dictionary
metamodel_parameters=df.to_dict()

print("Converted meta to dict")


fixed_parameters = {"cdcap": 0.0025}
hurrywaves_model = bluemath_tk.wrappers._base_wrappers.DummyModelWrapper(
templates_dir=templates_dir,
metamodel_parameters=metamodel_parameters,
fixed_parameters=fixed_parameters,
output_dir=output_dir,
)

print("Loaded model")


adjustedmmax
Converted meta to dict
Loaded model


In [9]:
hurrywaves_model.build_cases(mode="one_by_one") # one_by_one or all_combinations

In [10]:
hurrywaves_model.run_cases_bulk(
launcher="sbatch --array=0-10 /gpfs/work3/0/ai4nbs/hurry_wave/north_sea/04_modelruns/SENSITIVITY/Launcher/launch_sensitivity_december.sh")

Submitted batch job 15352018


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 64 CPU cores, 737280 MiB of memory and 4 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 11520 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1 GPUs, based on the number of CPUs, GPUs and the amount memory that you've requested.
